In [1]:
using JuMP, Gurobi, IPG

In [2]:
n = m = 2

X = [Strategies(-1, 0, 0), Strategies(-1, 0, 0)]
Π = [QuadraticPayoff(0, [2, 1]), QuadraticPayoff(0, [1, 2])]
;

In [3]:
x = [[0],[0]]
payoff(Π, x, 1)

0.0

# Step 1: Initialization

Compute one pure strategy for each player. Build sampled normal-form game.

In [4]:
function find_feasible_pure_strategy(Xp::Strategies, Πp::QuadraticPayoff)
    Ap, bp, Bp = Xp.Ap, Xp.bp, Xp.Bp

    # Create a new model to solve the feasibility problem
    m = Model(Gurobi.Optimizer)
    @variable(m, xp[1:np(Xp)])
    for i in 1:Bp  # TODO: there's probably a cleaner way to do this
        set_binary(xp[i])
    end
    @constraint(m, Ap * xp <= bp)
    @objective(m, Min, 0)  # Feasibility!

    set_silent(m)
    optimize!(m)

    return value.(xp)
end

function find_feasible_pure_profile(X::Vector{Strategies}, Π::Vector{QuadraticPayoff})
    return [find_feasible_pure_strategy(Xp, Πp) for (Xp, Πp) in zip(X, Π)]
end

x = find_feasible_pure_profile(X, Π)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca


2-element Vector{Vector{Float64}}:
 [0.0]
 [0.0]

In [5]:
x[1][1] = 10
x[2][1] = 10
x

2-element Vector{Vector{Float64}}:
 [10.0]
 [10.0]

In [6]:
# sample of strategies
S_X = [Vector{Vector{Float64}}() for p in 1:m]
# S_X[p,j,i] is the i-th dimension of the j-th strategy of player p

for p in 1:m
    # add to sample the new strategy
    push!(S_X[p], x[p])
end

In [7]:
# for p in 1:m
#     # add to sample the new strategy
#     push!(S_X[p], [1.0])
# end

# Step 2: Solve sampled game

Compute a solution to the normal-form game with the sampled strategies.

In [8]:
# TODO: making the polymatrix with "self-produced utilities" breaks the algorithms in 
# NormalGames. I don't know why this is, but it is mentioned that they assume so in a
# comment of function `GenerateGame`.

# function get_polymatrix(S_X::Vector{<:Vector{<:Vector{<:Real}}}, Π::Vector{QuadraticPayoff})
#     m = length(S_X)

#     dims = [length(S_X[p]) for p in 1:m]

#     polymatrix = Dict{Tuple{Integer, Integer}, Matrix{Float64}}()

#     # compute utility of each player `p` using strategy `i_p` against player `k` using strategy `i_k`
#     for p in 1:m
#         for k in 1:m
#             polymatrix[p,k] = zeros(length(S_X[p]), length(S_X[k]))

#             if k == p  # fills only the diagonal
#                 # TODO: this could be optimized by storing just the diagonal as well
#                 for i_p in 1:length(S_X[p])
#                     polymatrix[p,p][i_p,i_p] = IPG.bilateral_payoff(Π[p], p, S_X[p][i_p], p, S_X[p][i_p])
#                 end
#             else
#                 for i_p in 1:length(S_X[p])
#                     for i_k in 1:length(S_X[k])
#                         polymatrix[p,k][i_p,i_k] = IPG.bilateral_payoff(Π[p], p, S_X[p][i_p], k, S_X[k][i_k])
#                     end
#                 end
#             end
#         end
#     end

#     return polymatrix
# end

# polymatrix = get_polymatrix(S_X, Π)

In [27]:
function get_polymatrix(S_X::Vector{<:Vector{<:Vector{<:Real}}}, Π::Vector{QuadraticPayoff})
    m = length(S_X)

    dims = [length(S_X[p]) for p in 1:m]

    polymatrix = Dict{Tuple{Integer, Integer}, Matrix{Float64}}()

    # compute utility of each player `p` using strategy `i_p` against player `k` using strategy `i_k`
    for p in 1:m
        for k in 1:m
            polymatrix[p,k] = zeros(length(S_X[p]), length(S_X[k]))

            if k != p  # fills only the diagonal
                for i_p in 1:length(S_X[p])
                    for i_k in 1:length(S_X[k])
                        polymatrix[p,k][i_p,i_k] = (
                            IPG.bilateral_payoff(Π[p], p, S_X[p][i_p], k, S_X[k][i_k])
                            + IPG.bilateral_payoff(Π[p], p, S_X[p][i_p], p, S_X[p][i_p])
                        )
                    end
                end
            end
        end
    end

    return polymatrix
end

polymatrix = get_polymatrix(S_X, Π)

Dict{Tuple{Integer, Integer}, Matrix{Float64}} with 4 entries:
  (1, 2) => [0.0 -75.0; 25.0 -12.5]
  (1, 1) => [0.0 0.0; 0.0 0.0]
  (2, 2) => [0.0 0.0; 0.0 0.0]
  (2, 1) => [0.0 -50.0; 18.75 6.25]

In [28]:
using NormalGames

G = NormalGames.NormalGame(m, length.(S_X), polymatrix)

NormalGames.NormalGame(2, [2, 2], Dict((1, 2) => [0.0 -75.0; 25.0 -12.5], (1, 1) => [0.0 0.0; 0.0 0.0], (2, 2) => [0.0 0.0; 0.0 0.0], (2, 1) => [0.0 -50.0; 18.75 6.25]))

In [29]:
t, NE_utilities, NE_mixed = NormalGames.NashEquilibriaPNS(G,false,false,false)
# each element in NE_mixed is a mixed NE, represented as a vector of probabilities in the same shape as S_X

NE_mixed = NE_mixed[1]  # take the first NE

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca


2-element Vector{Vector{Float64}}:
 [0.0, 1.0]
 [0.0, 1.0]

In [30]:
σ = [IPG.DiscreteMixedStrategy(NE_mixed[p], S_X[p]) for p in 1:m]

2-element Vector{DiscreteMixedStrategy}:
 DiscreteMixedStrategy([1.0], [[5.0]])
 DiscreteMixedStrategy([1.0], [[2.5]])

# Step 3: Termination

Build a new strategy from the best reactions of each player.
Check if the new strategy is $\varepsilon$-close to the mixed NE.

In [31]:
function deviation_reaction(Π::Vector{QuadraticPayoff}, X::Vector{Strategies}, p::Integer, σ::Vector{DiscreteMixedStrategy})
    Πp = Π[p]
    Xp = X[p]

    model = Model(Gurobi.Optimizer)

    # the length of the strategy must match the strategies in the support of the profile
    @variable(model, xp[1:length(σ[p].supp[1])])

    @constraint(model, Xp.Ap * xp <= Xp.bp)

    # TODO: No idea why the following doesn't work
    # @objective(model, Max, sum([IPG.bilateral_payoff(Πp, p, xp, k, σ[k]) for k in 1:m]))

    obj = QuadExpr()
    for k in 1:m
        obj += IPG.bilateral_payoff(Πp, p, xp, k, σ[k])
    end
    @objective(model, Max, obj)

    set_silent(model)
    optimize!(model)

    return value.(xp)
end

deviation_reaction(Π, X, 1, σ)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca


1-element Vector{Float64}:
 1.25

In [32]:
function termination(Π::Vector{QuadraticPayoff}, X::Vector{Strategies}, σ::Vector{DiscreteMixedStrategy})
    for p in 1:m
        new_x_p = deviation_reaction(Π, X, p, σ)

        new_σ = copy(σ)
        new_σ[p] = IPG.DiscreteMixedStrategy([1], [new_x_p])
        if payoff(Π, σ, p) < payoff(Π, new_σ, p)
            return p, new_x_p
        end
    end

    return -1, nothing
end

p, xk = termination(Π, X, σ)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca


(1, [1.25])

# Step 4: Generation of next sampled game

Add the newly found strategy for player $p$ to `S_X` (sampled game), and close the loop.

In [33]:
push!(S_X[p], xk)

3-element Vector{Vector{Float64}}:
 [10.0]
 [5.0]
 [1.25]